In [ ]:
from datasets import load_dataset
data_dir = "../data/hf_dataset/"

dataset = load_dataset("imagefolder", data_dir=data_dir, drop_labels=False)

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
dataset['train'][0]

In [ ]:
from transformers import AutoImageProcessor
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
model_name ="microsoft/resnet-18"
image_processor = AutoImageProcessor.from_pretrained(model_name)


extractor = AutoFeatureExtractor.from_pretrained(model_name)

model = AutoModelForImageClassification.from_pretrained(model_name)

In [ ]:
# from datasets import load_dataset

# dataset = load_dataset("imagefolder", data_dir="/path/to/folder", drop_labels=False)

In [ ]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [ ]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
label2id, id2label

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

In [ ]:
dataset = dataset.with_transform(transforms)

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    output_dir="CIRCL_website_classifier",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
from transformers import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="./CIRCL_website_classifier", hub_model_id="Abhilashvj/CIRCL_website_classifier"
)

In [ ]:
from transformers import pipeline
from PIL import Image
image = Image.open('../data/test_website_images_25\\forum\\strong-spurious-plausible-database.png')
classifier = pipeline("image-classification", model="website_classifier")
classifier(image)